<h1> Assignment (Week 3) Capstone Project </h1><br><br>
<h1> Part I Retrieval and preparation of Toronto-Postal Code Data </h1>

<h3> After loading dependencies for this and later steps, <br>
The Beautiful-Soup Package was used to extract Data from Wikipedia. <br>
<bt>
Please keep in mind that there maybe issues with LXML-Parsing on some devices (it did on mine). <br>
This may require restarting your notebook after importing/sometimes installing/deinstalling the respective package <br>
<br>
Process: Irrelevant rows were removed from the data frame <br>
before editing the data frame (i.e. renaming empty Neighbourhood-Cells and grouping) <br> 
and finally printing the number of rows (i.e. relevant PostCodes)

In [ ]:
# Loading dependencies
from bs4 import BeautifulSoup # Used for wiki-scraping

import numpy as np # library to handle data in a vectorized manner
import requests

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

In [ ]:
# Loading required Toronto-Table from Wikipedia
wiki_url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&direction=prev&oldid=926287641'
wiki = requests.get(wiki_url).text
html_soup=BeautifulSoup(wiki,'html.parser')
sov_tables = html_soup.find('table', class_="wikitable sortable")
df = pd.read_html(str(sov_tables))
df = df[0]
df.head()

In [ ]:
# Remove rows without assigned Borough
df=df[df.Borough != 'Not assigned']


In [ ]:
# Replace 'Not Assigned' Neighbourhoods with corresponding 'Borough'-Entry
df.Neighbourhood=df.Neighbourhood.replace('Not assigned',df.Borough)

In [33]:
# Create and check new dataframe by grouping based on PostCode and join Neighbourhood entrys
df_Toronto=df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_Toronto.head(5)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<h2> Solution to Part I; Summary of changes to data frame </h2>

In [34]:
# Check Shape
df_Toronto.shape

(103, 3)

In [32]:
print('The grouped Data frame contains {} Rows (i.e. Postalcodes) as opposed to the non-grouped data frame, which contained {} rows'.format(df_Toronto.shape[0],df.shape[0]))

The grouped Data frame contains 103 Rows (i.e. Postalcodes) as opposed to the non-grouped data frame, which contained 211 rows


<h1> Part II Matching Postal-Codes to Location Data </h1>

In [ ]:
# Load CSV Data (as geocoder happened to be to unreliable)
import wget

url = 'https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv'
wget.download(url, 'Geospatial_Coordinates.csv')

print ('File downloaded and saved!')

In [ ]:
# Edit Coordinate-Table to allow mergin
Geospatial_Coordinates = pd.read_csv('Geospatial_Coordinates.csv')
Geospatial_Coordinates.columns = ['Postcode','Latitude','Longitude']

In [29]:
# Merge Coordinate Data with Neigbourhood/Toronto-Dataframe
df_ToCo = pd.merge(df_Toronto, Geospatial_Coordinates, on='Postcode')

<h2> Solution for Part II: Matched Postcodes to Location Data

In [31]:
# Check Result
df_ToCo.head(15)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
